# 简单二分+tfidf，中文模型

In [1]:
# 0. 导入库
import json
import os
import pandas as pd
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments


# 1. 读取数据集路径
human_path = "face2_zh_json/human/zh_unicode"
llm_path = "face2_zh_json/generated/zh_qwen2"

human_files = [os.path.join(human_path, f) for f in os.listdir(human_path)]
llm_files = [os.path.join(llm_path, f) for f in os.listdir(llm_path)]

print("✅ 人工文件:", human_files[:3])
print("✅ 千问文件:", llm_files[:3])

✅ 人工文件: ['face2_zh_json/human/zh_unicode\\news-zh.json', 'face2_zh_json/human/zh_unicode\\webnovel.json', 'face2_zh_json/human/zh_unicode\\wiki-zh.json']
✅ 千问文件: ['face2_zh_json/generated/zh_qwen2\\news-zh.qwen2-72b-base.json', 'face2_zh_json/generated/zh_qwen2\\webnovel.qwen2-72b-base.json', 'face2_zh_json/generated/zh_qwen2\\wiki-zh.qwen2-72b-base.json']


In [2]:
# 2. 加载人工数据
def load_human_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    df = pd.DataFrame(data)
    df["label"] = 0  # 人工标记为0
    return df[["output", "label"]]

human_df = pd.concat([load_human_data(f) for f in human_files], ignore_index=True)
print("👀 人工数据样例：")
print(human_df.head(3))

👀 人工数据样例：
                                              output  label
0  补贴后，变成了以积分、抽奖等形式为主的“暗补”。一组公开的数据显示，停补后的“滴滴打车”日均...      0
1  培训、投融资等方面有着巨大的合作空间，为积极推动我省与丹麦的友好交流，促进双边经贸投资合作，...      0
2                               环结你喜欢哪种呢？觉得不错，请点赞↓↓↓      0


In [3]:
# 3. 加载千问数据
def load_llm_data(file):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    outputs = [{"output": v, "label": 1} for k, v in data["output"].items()]
    return pd.DataFrame(outputs)

llm_df = pd.concat([load_llm_data(f) for f in llm_files], ignore_index=True)
print("🤖 千问生成数据样例：")
print(llm_df.head(3))

🤖 千问生成数据样例：
                                              output  label
0  补贴主要针对司机端。\n记者昨日从快的打车获悉，针对司机端的补贴将在今天正式实施：早上7点至...      1
1  合作以及教育交流等方面有着广阔的合作空间，5月20日，省委统战部、省环保厅和省外事办在贵阳联...      1
2  环绕希望对大家有用！\n这9个基本功，99%的家长不会教孩子！\n这9个基本功，99%的家长...      1


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# 1. 载入数据
all_df = pd.concat([human_df, llm_df], ignore_index=True)
dataset = Dataset.from_pandas(all_df.rename(columns={"output": "text"}))

In [7]:
#model_name = "bert-base-chinese"
model_name = "hfl/chinese-bert-wwm-ext"         # 比较大
# 2. 数据预处理：分词与向量化
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

# 使用map方法进行批处理分词
dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [8]:
# 3. 切分训练集和验证集
train_dataset, eval_dataset = dataset.train_test_split(test_size=0.1).values()

# 4. 设置模型和训练参数
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# 5. 设置 Trainer 参数
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",  # 修改这里
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    do_train=True,
    do_eval=True,
    report_to="none",  # 关闭日志报告（如 TensorBoard）
    dataloader_num_workers=4,  # 多线程加载数据，提高数据加载效率
    fp16=False,  # CPU 禁用混合精度
    save_total_limit=2,  # 最多保存 2 个 checkpoint
    load_best_model_at_end=True,  # 加载最优模型
    metric_for_best_model="eval_loss",  # 根据评估 loss 判断最优模型
    greater_is_better=False
)
# 6. Trainer 训练
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

C:\Users\86195\AppData\Local\Temp\ipykernel_26040\1821734847.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
# 训练模型
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.502700,0.544023
2,0.405900,1.017501
3,0.217200,1.438685


d:\86195\Anaconda\envs\Pytorch_py310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
d:\86195\Anaconda\envs\Pytorch_py310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=10125, training_loss=0.37735411377895023, metrics={'train_runtime': 27680.4464, 'train_samples_per_second': 2.926, 'train_steps_per_second': 0.366, 'total_flos': 5334839758479360.0, 'train_loss': 0.37735411377895023, 'epoch': 3.0})

In [12]:
# ✅ 保存最终模型和 tokenizer
save_path = "./saved_model"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)
print(f"✅ 模型已保存到 {save_path}")

# 9. 评估模型
results = trainer.evaluate()
print("评估结果：")
print(results)

# 10. 预测并输出分类报告
predictions = trainer.predict(eval_dataset)
pred_labels = predictions.predictions.argmax(axis=-1)
from sklearn.metrics import classification_report, roc_auc_score
print("分类报告：")
print(classification_report(eval_dataset["label"], pred_labels))

# 计算 AUC
roc_auc = roc_auc_score(eval_dataset["label"], predictions.predictions[:, 1])
print(f"AUC 分数: {roc_auc:.4f}")

✅ 模型已保存到 ./saved_model


d:\86195\Anaconda\envs\Pytorch_py310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


评估结果：
{'eval_loss': 0.5440232157707214, 'eval_runtime': 255.0984, 'eval_samples_per_second': 11.76, 'eval_steps_per_second': 1.47, 'epoch': 3.0}
分类报告：
              precision    recall  f1-score   support

           0       0.88      0.50      0.64      1534
           1       0.64      0.93      0.76      1466

    accuracy                           0.71      3000
   macro avg       0.76      0.71      0.70      3000
weighted avg       0.76      0.71      0.70      3000

AUC 分数: 0.8447


In [2]:
import os
import pandas as pd
import random
from sklearn.metrics import classification_report, roc_auc_score
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments
)
from datasets import Dataset
import torch

def load_texts_from_folders(base_folder, subfolders):
    texts = []
    for subfolder in subfolders:
        folder_path = os.path.join(base_folder, subfolder)
        file_list = [f for f in os.listdir(folder_path) if f.endswith(".txt")]
        file_list = sorted(file_list, key=lambda x: int(x.split(".")[0]))
        for file_name in file_list:
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, "r", encoding="utf-8") as f:
                texts.append(f.read())
    return texts

# 加载人类写的文本
human_texts = load_texts_from_folders("ghostbuster-data/essay", ["human"])

# 加载所有GPT生成的文本（从三个不同的子目录）
gpt_texts = load_texts_from_folders("ghostbuster-data/essay", ["gpt", "gpt_writing", "gpt_semantic"])

# 显示一些样本
print(f"Loaded {len(human_texts)} human texts and {len(gpt_texts)} GPT texts")
print("\nSample human text:\n", human_texts[299][:500])
print("\nSample GPT text:\n", gpt_texts[2990][:500])

Loaded 1000 human texts and 3000 GPT texts

Sample human text:
 Environmental Factor
Ensuring the safety of a newborn child becomes the primary and most important task of parents. Especially in the first year of life, the child is unable to protect himself from damaging environmental factors. Examples of such harmful effects may be air and water pollution, excessive sun exposure, or dust and chemicals. This work analyzes the harmful environmental factors for a child under one year old and offers health promotion to protect newborns from harm.
Sources emphasi

Sample GPT text:
 Introduction:
Dyslexia is a learning disorder that affects numerous students, making it challenging for them to read, spell, and write. With its profound impact on academic performance, understanding the effects of dyslexia on children and exploring specialized interventions becomes crucial to ensure their success in school.
Body Paragraphs:
Effects of Dyslexia:
Dyslexia can have various effects on children, both

In [3]:
# 2. 构建 DataFrame
texts = human_texts + gpt_texts
labels = [0] * len(human_texts) + [1] * len(gpt_texts)  # 0: Human, 1: GPT
combined = list(zip(texts, labels))
random.shuffle(combined)
texts, labels = zip(*combined)
df = pd.DataFrame({"text": texts, "label": labels})

In [4]:
# 3. 转换为 HuggingFace Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]
print(len(train_dataset))

3600


In [5]:

# 4. 加载 tokenizer 和模型
model_name = "roberta-base"  
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize, batched=True)
eval_dataset = eval_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [6]:
# 5. 加载模型
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# 6. 设置 TrainingArguments（自动保存模型）
training_args = TrainingArguments(
    output_dir="./bert-eng-results",
    eval_strategy="epoch",
    save_strategy="epoch",  # 自动保存最佳模型
    save_total_limit=2,     # 最多保留2个模型快照
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./bert-eng-logs",
    logging_steps=50,
    do_train=True,
    do_eval=True,
    report_to="none",
    dataloader_num_workers=4,
    fp16=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # 根据最小loss保存最佳模型
    greater_is_better=False,
)

# 7. 定义 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

C:\Users\86195\AppData\Local\Temp\ipykernel_33268\1934617829.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [8]:
# 8. 训练模型
trainer.train()

# 9. 评估模型
results = trainer.evaluate()
print("评估结果：")
print(results)

# 10. 预测与分类报告
predictions = trainer.predict(eval_dataset)
pred_labels = predictions.predictions.argmax(axis=-1)
print("分类报告：")
print(classification_report(eval_dataset["label"], pred_labels))
roc_auc = roc_auc_score(eval_dataset["label"], predictions.predictions[:, 1])
print(f"AUC 分数: {roc_auc:.4f}")

d:\86195\Anaconda\envs\Pytorch_py310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,0.048200,0.000336
2,0.000200,0.012816
3,0.000100,0.050201


d:\86195\Anaconda\envs\Pytorch_py310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
d:\86195\Anaconda\envs\Pytorch_py310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
d:\86195\Anaconda\envs\Pytorch_py310\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


评估结果：
{'eval_loss': 0.18355199587531388, 'eval_runtime': 161.9234, 'eval_samples_per_second': 2.47, 'eval_steps_per_second': 0.309, 'epoch': 3.0}
分类报告：
              precision    recall  f1-score   support

           0       0.75      0.83      0.79        96
           1       0.94      0.92      0.93       304

    accuracy                           0.89       400
   macro avg       0.85      0.88      0.86       400
weighted avg       0.90      0.89      0.89       400

AUC 分数: 0.9320
